<a href="https://colab.research.google.com/github/doaa450/Customer-lifetime-value/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pyngrok
from pyngrok import ngrok
from flask import Flask

ngrok_token = 'my key'
port = 5000
ngrok.set_auth_token('my key')

from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import pickle
import joblib
import shap
import base64
import io

scaler_xgb=pickle.load(open('scaler_xgb.pickle', 'rb'))
best_model=pickle.load(open('best_model.pickle', 'rb'))

scaler_kmeans=pickle.load(open('scaler_kmeans.pickle', 'rb'))
pca=pickle.load(open('pca.pickle', 'rb'))
model=pickle.load(open('model.pickle', 'rb'))

shap.initjs()
explainer = joblib.load(filename="explainer.bz2")

app = Flask(__name__)

@app.route('/',methods=['Get'])
def Home():
    return render_template ('index.html')

@app.route('/Predict', methods=['GET'])
def predict():
    all_data = request.args
    Recency = float(all_data['Recency'])
    Frequency = float(all_data['Frequency'])
    Monetary_Sum = float(all_data['Monetary_Sum'])
    Monetary_Min = float(all_data['Monetary_Min'])
    Monetary_Max = float(all_data['Monetary_Max'])
    Monetary_Mean = float(all_data['Monetary_Mean'])
    Monetary_std = float(all_data['Monetary_std'])
    Quantity = float(all_data['Quantity'])
    RecencyScore = int(all_data['RecencyScore'])
    FrequencyScore = int(all_data['FrequencyScore'])
    MonetaryScore = int(all_data['MonetaryScore'])
    Segment = int(all_data['Segment'])
    Unique_Products_Purchased = int(all_data['Unique_Products_Purchased'])
    Cancellation_Frequency = float(all_data['Cancellation_Frequency'])
    Cancellation_Rate = float(all_data['Cancellation_Rate'])


    features=[Recency,Frequency,Monetary_Sum,Monetary_Min,Monetary_Max,Monetary_Mean,Monetary_std,Quantity,RecencyScore,FrequencyScore,MonetaryScore,Segment,Unique_Products_Purchased,Cancellation_Frequency,Cancellation_Rate]

    columns =['Recency','Frequency','Monetary_Sum','Monetary_Min','Monetary_Max','Monetary_Mean','Monetary_std','Quantity','RecencyScore','FrequencyScore','MonetaryScore','Segment','Unique_Products_Purchased','Cancellation_Frequency','Cancellation_Rate']

    features_df=pd.DataFrame([features],columns=columns)

    # List of columns that don't need to be scaled
    columns_to_exclude = ['RecencyScore', 'FrequencyScore', 'MonetaryScore','Segment']

    # List of columns that need to be scaled
    columns_to_scale = features_df.columns.difference(columns_to_exclude)


    # Applying the scaler to the necessary columns in the dataset
    features_df[columns_to_scale] = scaler_xgb.transform(features_df[columns_to_scale])



    Total_Sales_3M = best_model.predict(features_df)

    #Shap value
    columns_1=features_df.columns
    shap_values2 = explainer.shap_values(features_df)
    shap_img = io.BytesIO()
    shap.plots.force( explainer.expected_value,shap_values2,columns_1,matplotlib=True,show = False,figsize=(35,5)).savefig(shap_img, bbox_inches="tight", format = 'png')
    shap_img.seek(0)
    shap_url = base64.b64encode(shap_img.getvalue()).decode()

    #Kmeans
    features_2=features+[Total_Sales_3M]
    columns_2=columns+['Total_Sales_3M']
    profit_df=pd.DataFrame([features_2],columns=columns_2)
    # List of columns that don't need to be scaled
    columns_to_exclude = ['RecencyScore', 'FrequencyScore', 'MonetaryScore','Segment']

    # List of columns that need to be scaled
    columns_to_scale = profit_df.columns.difference(columns_to_exclude)
    # Applying the scaler to the necessary columns in the dataset
    profit_df[columns_to_scale] = scaler_kmeans.transform(profit_df[columns_to_scale])

    df_segm_pca = pca.transform(profit_df)
    segm_kmeans_pca = model.predict(df_segm_pca[:,:2])
    clu_num=segm_kmeans_pca[0]
    Total_Sales_3M=round(float(Total_Sales_3M),0)
    clu_num=clu_num.item()

   # render result
    return render_template ('index.html',Total_Sales_3M=Total_Sales_3M,clu_num=clu_num,url_1 = shap_url)


if __name__ == "__main__":
  try:
    public_url = ngrok.connect(port).public_url
    print(public_url)
    app.run(port=port)
  finally:
    ngrok.disconnect(public_url=public_url)